In [ ]:
pip install upsetplot

In [ ]:
pip install pROC

In [ ]:
pip install iterative-stratification

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from upsetplot import UpSet
from upsetplot import plot
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import sklearn
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score
#from delong import auc_ci
import scipy.stats

In [ ]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2, n-1)
    return m, m-h, m+h

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root = "/content/drive/MyDrive/CCIDS"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv = pd.read_csv('/content/drive/MyDrive/CCIDS/210807_multilabel_binCount_32.csv')
clf = RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
auc = []; acc = []; rec = []; pre = [];
                                                                                
skf = MultilabelStratifiedKFold(n_splits=3, shuffle=True, random_state=42)

X = csv.drop(['ID', 'DrD2', 'SSTR2', 'SSTR5'], axis=1).values
y = csv[['DrD2', 'SSTR2', 'SSTR5']].values
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = sklearn.base.clone(clf); clf.fit(X_train, y_train)
    skf_pred = clf.predict(X_test); skf_proba = clf.predict_proba(X_test)
    
    DrD2 = []; SSTR2 = []; SSTR5 = [];
    for i in range(len(skf_pred)) : 
      DrD2.append(skf_pred[i,0])
      SSTR2.append(skf_pred[i,1])
      SSTR5.append(skf_pred[i,2])

    auc.append(roc_auc_score(y_test, skf_pred, average='weighted'))
    acc.append(accuracy_score(y_test, skf_pred))
    rec.append(recall_score(y_test, skf_pred, average='weighted'))
    pre.append(precision_score(y_test, skf_pred, average='weighted'))

In [121]:
train, test = train_test_split(csv, stratify=csv[['DrD2', 'SSTR2', 'SSTR5']], test_size=0.4, random_state=42)
# train.to_csv('210807_multilabel_binCount_train.csv', mode='w', header=True, index=False)
# test.to_csv('210807_multilabel_binCount_test.csv', mode='w', header=True, index=False)

X_train = train.drop(['ID', 'DrD2', 'SSTR2', 'SSTR5'], axis=1).values
y_train = train[['DrD2', 'SSTR2', 'SSTR5']].values
X_test = test.drop(['ID', 'DrD2', 'SSTR2', 'SSTR5'], axis=1).values
y_test = test[['DrD2', 'SSTR2', 'SSTR5']].values

In [122]:
clf = sklearn.base.clone(clf); clf.fit(X_train, y_train)
test_pred = clf.predict(X_test); test_proba = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_test, test_pred, average='weighted')
test_acc = accuracy_score(y_test, test_pred)
test_rec = recall_score(y_test, test_pred, average='weighted')
test_pre = precision_score(y_test, test_pred, average='weighted')

In [123]:
train_dict = {'train_auc' : np.mean(auc), 'train_auc_95lower' : mean_confidence_interval(auc)[1], 'train_auc_95upper' : mean_confidence_interval(auc)[2], 'train_acc' : np.mean(acc), 'train_rec' : np.mean(rec), 'train_pre' : np.mean(pre)}

In [124]:
test_dict = {'test_auc' : test_auc,'test_acc' : test_auc, 'test_rec' : test_rec, 'test_pre' : test_pre}

In [125]:
train_df = pd.DataFrame(train_dict.values(), train_dict.keys()).T
test_df = pd.DataFrame(test_dict.values(), test_dict.keys()).T

In [126]:
df = pd.concat([train_df, test_df], axis=1)
df

,train_auc,train_auc_95lower,train_auc_95upper,train_acc,train_rec,train_pre,test_auc,test_acc,test_rec,test_pre
0,0.566746,0.529595,0.603897,0.206439,0.545065,0.568418,0.624864,0.624864,0.62069,0.622206


여기부터 multilabel
